In [273]:
import pandas as pd 
data = pd.read_csv('CE802_Ass_2019_Data.csv').sample(frac=1)
test_features = pd.read_csv('CE802_Ass_2019_Test.csv').drop("Class", axis=1)

In [274]:
split = int(len(data) * 0.15)
validation = data[0:split]
train = data[split:]

In [275]:
train_labels = train["Class"]
train_features = train.drop("Class", axis=1)
validation_labels = validation["Class"]
validation_features = validation.drop("Class", axis=1)

In [276]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def imputeF20(X, method='none'):
    if method == 'none':
        return pd.DataFrame(X)
    if method == 'drop':
        X = X.drop("F20",axis=1).values
        return pd.DataFrame(X)
    if method == 'constant':
        imp = SimpleImputer(strategy='constant')
    if method == 'mean':
        imp = SimpleImputer(strategy='mean')
    if method == 'knn':
        imp = KNNImputer(n_neighbors=5)

    imp.fit(X)
    return pd.DataFrame(imp.transform(X))

def scale(X, method='none'):
    if method == 'none':
        return pd.DataFrame(X)
    if method == 'norm':
        scaler = MinMaxScaler()
    if method == 'std':
        scaler = StandardScaler()
    
    scaler.fit(X)
    return pd.DataFrame(scaler.transform(X))

In [319]:
imputation_method = 'knn'
scaling_method = 'norm'

train_features_procesed = imputeF20(train_features, imputation_method)
train_features_procesed = scale(train_features_procesed, scaling_method )

validation_features_procesed = imputeF20(validation_features, imputation_method)
validation_features_procesed = scale(validation_features_procesed, scaling_method )

In [411]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
  {'max_depth': [5,20,30,50,100,200]},
 ]
tree = DecisionTreeClassifier()
# tree.fit(train_features_procesed, train_labels);

grid_search = GridSearchCV(tree, param_grid, cv=3, scoring="f1", return_train_score=True)

grid_search.fit(train_features_procesed, train_labels)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [5, 20, 30, 50, 100, 200]}],
          

{'max_depth': 100}

In [393]:
# preds = tree.predict(validation_features_procesed)

In [394]:
from sklearn.model_selection import cross_val_score
cross_val_score(tree, train_features_procesed, train_labels, cv=3, scoring='f1')

array([0.52348993, 0.512     , 0.23255814])

In [396]:
from sklearn.model_selection import cross_val_predict
cross_val_predict(tree, train_features_procesed, train_labels, cv=3)

array([ True, False, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True, False, False,  True, False, False, False,
       False, False,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True, False, False,  True,  True,
       False,  True, False,  True, False,  True,  True,  True,  True,
        True, False,  True,  True, False,  True, False,  True, False,
        True,  True, False, False, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True, False,  True,
       False, False,  True, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True,  True,  True,
       False, False, False,  True,  True,  True,  True, False,  True,
        True, False,  True,  True, False, False, False,  True,  True,
       False,  True,  True, False, False,  True, False, False,  True,
        True, False,

In [ ]:
# Implementar el grid search que se borro
# Implemtar Pruning
# Hacer otros clfs, minimo 2
# Comparar